In [1]:
!pip install pytorch_lightning

Looking in indexes: http://pypi.douban.com/simple, http://mirrors.aliyun.com/pypi/simple/, https://pypi.tuna.tsinghua.edu.cn/simple/, http://pypi.mirrors.ustc.edu.cn/simple/
  Using cached https://mirrors.tuna.tsinghua.edu.cn/pypi/web/packages/26/b1/f0cbbabeceab4470bde31c025e4ce22084ff446a3f022100f032db9ea88b/pytorch_lightning-2.0.6-py3-none-any.whl (722 kB)


In [ ]:
import torch
import pytorch_lightning

import os
import sys
import datetime
import torchkeras

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('/data/snlp/zhangjl/datas/ctr/eat_pytorch_datasets/covid-19.csv', 
                 sep='\t')
df.plot(x='date', y = ["confirmed_num","cured_num","dead_num"],figsize=(10,6))

In [ ]:
dfdata = df.set_index("date")
dfdiff = dfdata.diff(periods=1).dropna()
dfdiff = dfdiff.reset_index("date")

dfdiff.plot(x = "date",y = ["confirmed_num","cured_num","dead_num"],figsize=(10,6))
plt.xticks(rotation=60)
dfdiff = dfdiff.drop("date",axis = 1).astype("float32")

In [ ]:
import torch
from torch import nn 
from torch.utils.data import Dataset, DataLoader, TensorDataset

WINDOW_SIZE = 8
class Covid19DataSet(Dataset):
    def __len__(self):
        return len(dfdiff) - WINDOW_SIZE
    
    def __getitem__(self, i):
        x = dfdiff.loc[i:i+WINDOW_SIZE-1,:]
        feature = torch.tensor(x.values)
        y = dfdiff.loc[i+WINDOW_SIZE,:]
        label = torch.tensor(y.values)
        return (feature,label)
    
ds_train = Covid19DataSet()
dl_train = DataLoader(ds_train, batch_size=38)

for features, labels in dl_train:
    break

dl_val = dl_train

In [ ]:
import torch
from torch import nn 
import importlib
import torchkeras

class Block(nn.Module):
    def __init__(self):
        super(Block, self).__init__()

    def forward(self, x, x_input):
        x_out = torch.max((1 +x) *x_input[:,-1,:], torch.tensor(0.0))
        return x_out
    
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.lstm = nn.LSTM(input_size=3, hidden_size=3, num_layers =5, batch_first=True)
        self.linear = nn.Linear(3, 3)
        self.block = Block()

    def forward(self, x_input):
        x = self.lstm(x_input)[0][:, -1, :]
        x = self.linear(x)
        y = self.block(x, x_input)
        return y
net = Net()
print(net)